# Visualizador de creación de paradas y nodos

Este notebook es una herramienta para crear a partir de los recorridos de las lineas un dataset de paradas. A partir de cada linea se interpola una parada cada determinada cantidad de metros (definda por el parámetro `stops_distance` que debe estar como atributo en el geojson de los recorridos). A la hora de armar los recorridos como un grafo, y permitir transbordo dentro de ramales en una misma linea, se agregan paradas en mismo nodo. El criterio es agregar todas las paradas que esten a una determinada distancia entre sí. Este parámetro se fija parada cada `id_linea` en `line_stops_buffer` (que también debe estar como atributo en el geojson de los recorridos).

Este notebook sigue paso a paso este proceso y permite tunear el parámetro `line_stops_buffer` para cada linea, volviendo a guardar el resultado en el geojson de los recorridos.

El dataset de recorridos disponible en el repositorio es usado como ejemplo.
___

This notebook is a tool to create a dataset of stops from the routes of the lines. From each line, a stop is interpolated every certain number of meters (defined by the `stops_distance` parameter, which must be an attribute in the geojson of the routes). When building the routes as a graph, and allowing transfers within branches on the same line, stops are added at the same node. The criterion is to add all the stops that are at a certain distance from each other. This parameter sets each `id_linea` stop in `line_stops_buffer` (which must also be an attribute in the geojson of the routes).

This notebook follows this process step by step and allows you to tune the `line_stops_buffer` parameter for each line, saving the result back to the geojson of the tracks.

The track dataset available in the repository is used as an example.

In [ ]:
import geopandas as gpd
import os
from urbantrips.carto import stops
from urbantrips.utils import utils

Ingresar el directorio donde fue clonado el repositorio de `urbantrips` para que lea correctamente el archivo de configs.
___

Enter the directory where the `urbantrips` repository was cloned so that it reads the configs file correctly.

In [ ]:
URBANTRIPS_PATH = "[PATH]"

os.chdir(URBANTRIPS_PATH)

configs = utils.leer_configs_generales()
geojson_path = os.path.join(URBANTRIPS_PATH,"data","data_ciudad",configs['recorridos_geojson'])


In [ ]:
gdf = gpd.read_file(geojson_path)
gdf

A partir de los recorridos y el parámetro presente en `stops_distance` en el dataset para cada linea/ramal, se crearon las siguientes paradas.

____

From the routes and the parameter present in `stops_distance` in the dataset for each line/branch, the following stops were created.

In [ ]:
stops_gdf = stops.create_line_stops_equal_interval(geojson_path)
stops_gdf

Se puede chequear para cada linea la ubicación de las paradas interpoladas

___

You can check for each line the location of the interpolated stops

In [ ]:
stops_gdf.explore(column = 'id_linea', tiles="Stamen Toner", marker_kwds = {'radius':10}, cmap='Set2')

En esta viz puede chequear el orden de las paradas en el ramal para ver que sea incremental y continuo.

____

In this viz you can check the order of the stops in the branch to see that it is incremental and continuous.

In [ ]:
id_ramal = 0
stops_gdf.query(f"id_ramal == {id_ramal}").explore(column = 'order', tiles="Stamen Toner", marker_kwds = {'radius':10})

Ahora se puede tunear el parámetro `line_stops_buffer` para cada linea. 

Lo ideal es que no se hayan eliminado demasiadas paradas ni que queden demasiadas superpuestas. 

Si hay muy pocas paradas y son demasiadas las que fueron fusionadas en un solo `node_id` entonces utilizar un 
`line_stops_buffer` más conservador, de menor valor.

Si son demasiadas paradas y se desea fusionar algunas más, elegir un `line_stops_buffer` mayor.

Para la linea `143` con 2 ramales (que sigue el recorrido de la linea 12 de CABA) ligeras diferencias en el recorrido hace que se interpolen paradas demasiado cerca entre si para los dos ramales. Si se utiliza `line_stops_buffer = 50` estan quedarán duplicadas, pero si se utiliza `line_stops_buffer = 100` se fusionaran en una sola parada, en especial donde ambos ramales se superponen.

___
You can now tune the `line_stops_buffer` parameter for each line.

Ideally, not too many stops have been removed or too many overlapping ones.

If there are too few stops and too many were merged into a single `node_id` then use a
`line_stops_buffer` more conservative, lower value.

If there are too many stops and you want to merge some more, choose a larger `line_stops_buffer`.

For line `143` with 2 branches (which follows the route of line 12 of CABA) slight differences in the route mean that stops are interpolated too close to each other for the two branches. If `line_stops_buffer = 50` is used they will be duplicated, but if `line_stops_buffer = 100` is used they will be merged into a single stop, especially where both branches overlap.

In [ ]:
# Elegir linea y parametros
id_linea = 143
line_stops_buffer = 100

line_stops_gdf = stops_gdf.loc[stops_gdf.id_linea == id_linea,:]
line_stops_gdf.loc[:,['line_stops_buffer']] = line_stops_buffer

# Agregar a node_ide
stops_with_node_id = stops.create_node_id(line_stops_gdf)

# GDF para visualizar
geometry = gpd.points_from_xy(stops_with_node_id['x'], stops_with_node_id['y'], crs= 'EPSG:4326')
stops_with_node_id_gdf = gpd.GeoDataFrame(stops_with_node_id,geometry=geometry,crs = f"EPSG:4326")
stops_with_node_id_gdf.explore(color = 'red',tiles="Stamen Toner", marker_kwds = {'radius':10})

Si el parámetro es correcto se puede agregar al dataset y guardarlo en el mismo geojson

___

If the parameter is correct, it can be added to the dataset and saved in the same geojson

In [ ]:
geojson_data.loc[geojson_data.id_linea == id_linea,['line_stops_buffer']] = line_stops_buffer
geojson_data.to_file(geojson_path, driver='GeoJSON')

Una vez que el parámetro `line_stops_buffer` ha sido satisfactoriamente agregado para cada `id_linea`, puede correr `stops.create_temprary_stops_csv_with_node_id(geojson_path)` que dejará guardado el dataset de paradas con orden y `node_id`. Si lo desea puede agregar otro set de paradas que se haya confeccionado manualmente, pero debe tener la misma esctructura. Esto puede ser util para cuando se decide crear de modo manual un dataset de estas características para modos como el metro o el tren. 

___

Once the `line_stops_buffer` parameter has been successfully added for each `line_id`, you can run `stops.create_temprary_stops_csv_with_node_id(geojson_path)` which will save the stops dataset with order and `node_id`. If you wish, you can add another set of stops that has been created manually, but it must have the same structure. This can be useful when it is decided to manually create a dataset of these characteristics for modes such as the subway or the train.
